In [1]:

import requests
import pandas as pd
import numpy as np

from dotenv import load_dotenv
import os
import json
from pandas.io.json import json_normalize

import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
import seaborn as sns
pd.reset_option('display.max_rows')



In [2]:
# Cargar el archivo .env
load_dotenv()

# Acceder al valor de una variable
api_key = os.getenv("API_KEY")

folder_path = "data/IRON/Final"


In [103]:
df_test = pd.read_json("data/IRON/Timeline/test.json")


In [104]:
df_test["metadata"]["matchId"]

'LA2_1308805366'

In [85]:
#Esto es para un archivo json

df = pd.read_json('data/IRON/Timeline/test.json')
df_loop = pd.DataFrame(df["info"]["frames"][15]["participantFrames"]).transpose()[["championStats", "damageStats", "jungleMinionsKilled", "minionsKilled", "totalGold", "xp"]].reset_index(drop=True)
#df_loop = df_loop.transpose().drop(columns=["championStats","currentGold","goldPerSecond","participantId","position","timeEnemySpentControlled","level"])
#df_loop.reset_index(drop=True, inplace=True)
damage_stats = df_loop["damageStats"]
champion_stats = df_loop["championStats"]
damage_stats_expanded = pd.json_normalize(damage_stats)
champion_stats_expanded = pd.json_normalize(champion_stats)
#df_completo = pd.concat([df_loop, damage_stats_expanded, champion_stats_expanded], axis=1)
damage_stats_expanded = damage_stats_expanded.reset_index(drop=True)
champion_stats_expanded = champion_stats_expanded.reset_index(drop=True)

combined_df = df_loop.join(champion_stats_expanded)
combined_df = combined_df.join(damage_stats_expanded).drop(columns=["championStats", "damageStats"])


df_loop

,championStats,damageStats,jungleMinionsKilled,minionsKilled,totalGold,xp
0,"{'abilityHaste': 0, 'abilityPower': 15, 'armor...","{'magicDamageDone': 29266, 'magicDamageDoneToC...",0,81,5567,8821
1,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...","{'magicDamageDone': 21287, 'magicDamageDoneToC...",69,2,5415,5186
2,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...","{'magicDamageDone': 3304, 'magicDamageDoneToCh...",0,95,4324,6550
3,"{'abilityHaste': 0, 'abilityPower': 20, 'armor...","{'magicDamageDone': 1576, 'magicDamageDoneToCh...",4,109,7532,5713
4,"{'abilityHaste': 0, 'abilityPower': 53, 'armor...","{'magicDamageDone': 5947, 'magicDamageDoneToCh...",0,25,4546,3966
5,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...","{'magicDamageDone': 4200, 'magicDamageDoneToCh...",0,102,4929,6556
6,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...","{'magicDamageDone': 21146, 'magicDamageDoneToC...",76,13,5471,5626
7,"{'abilityHaste': 0, 'abilityPower': 80, 'armor...","{'magicDamageDone': 42542, 'magicDamageDoneToC...",1,90,4558,6457
8,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...","{'magicDamageDone': 138, 'magicDamageDoneToCha...",0,35,3492,4091
9,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...","{'magicDamageDone': 0, 'magicDamageDoneToChamp...",0,29,3462,4742


In [105]:
#Para leer todos los archivos JSON de una carpeta

def combine_dataframes(json_folder):
    # Obtener la lista de archivos JSON en la carpeta
    json_files = [file for file in os.listdir(json_folder) if file.endswith('.json')]

    # Crear una lista para almacenar los DataFrames de cada archivo
    dfs = []
    
    

    # Recorrer cada archivo JSON y combinar los datos en un DataFrame
    for json_file in json_files:
        
        # Reiniciar el contador para cada archivo JSON
        player_id_counter = 0
        
        json_path = os.path.join(json_folder, json_file)
        df = pd.read_json(json_path)

        # Verificar la estructura del archivo JSON
        if "info" in df and "frames" in df["info"]:
            frames = df["info"]["frames"]
            if len(frames) > 15 and "participantFrames" in frames[15]:
                participant_frames = frames[15]["participantFrames"]
                if participant_frames:
                    df_loop = pd.DataFrame(participant_frames).transpose()[["championStats", "damageStats", "jungleMinionsKilled", "minionsKilled", "totalGold", "xp"]].reset_index(drop=True)
                    damage_stats = df_loop["damageStats"]
                    champion_stats = df_loop["championStats"]
                    damage_stats_expanded = pd.json_normalize(damage_stats)
                    champion_stats_expanded = pd.json_normalize(champion_stats)
                    damage_stats_expanded = damage_stats_expanded.reset_index(drop=True)
                    champion_stats_expanded = champion_stats_expanded.reset_index(drop=True)

                    combined_df = df_loop.join(champion_stats_expanded)
                    combined_df = combined_df.join(damage_stats_expanded).drop(columns=["championStats", "damageStats"])
                    
                    # Agregar la columna "player_id" con valores secuenciales para cada archivo JSON
                    combined_df["player_id"] = range(player_id_counter, player_id_counter + len(combined_df))
                    
                    # Incrementar el contador de player_id para el siguiente archivo JSON
                    player_id_counter += len(combined_df)
                    
                    # Obtener el valor de matchId del archivo JSON
                    match_id = df["metadata"]["matchId"]
                    
                    # Repetir el valor de matchId en las 10 filas del DataFrame actual
                    combined_df["matchId"] = match_id

                    dfs.append(combined_df)  # Agregar el DataFrame combinado a la lista

    # Combinar todos los DataFrames en uno solo
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

# Ruta de la carpeta que contiene los archivos JSON
json_folder = 'data/IRON/Timeline/Partidas'

# Llamar a la función para combinar los DataFrames
combined_df = combine_dataframes(json_folder)




In [ ]:
#usar esto para leer el archivo de csv:

In [106]:
combined_df

,jungleMinionsKilled,minionsKilled,totalGold,xp,abilityHaste,abilityPower,armor,armorPen,armorPenPercent,attackDamage,...,physicalDamageDoneToChampions,physicalDamageTaken,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken,player_id,matchId
0,4,87,5753,7145,0.0,0,121,0,0,183,...,3375,4744,56079,4820,5798,8492,1444,0,0,LA2_1304858815
1,60,8,5759,5296,0.0,0,73,0,10,164,...,5737,6219,64422,6218,9646,14786,271,20,1,LA2_1304858815
2,0,78,4508,6734,0.0,112,64,0,0,82,...,446,1828,41228,5862,8329,26,26,423,2,LA2_1304858815
3,0,75,6480,5214,0.0,0,63,0,0,173,...,7203,2135,41659,7888,3698,4099,375,116,3,LA2_1304858815
4,0,24,4746,4606,0.0,0,127,0,0,92,...,1344,3110,13656,3529,5639,3098,0,266,4,LA2_1304858815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47045,4,87,4570,7947,0.0,0,75,0,0,153,...,990,5530,39856,1792,5935,4955,240,405,5,LA2_1253811110
47046,60,16,5621,5078,0.0,0,72,0,0,155,...,2852,7352,72486,5316,9749,13369,245,237,6,LA2_1253811110
47047,4,73,6166,6003,0.0,0,61,0,0,196,...,7948,5402,42117,8322,8476,374,374,597,7,LA2_1253811110
47048,8,78,4723,4832,0.0,0,58,0,0,129,...,3325,2746,43180,3998,5023,398,3,0,8,LA2_1253811110


In [107]:
df_trabajo = combined_df.copy()
df_trabajo

,jungleMinionsKilled,minionsKilled,totalGold,xp,abilityHaste,abilityPower,armor,armorPen,armorPenPercent,attackDamage,...,physicalDamageDoneToChampions,physicalDamageTaken,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken,player_id,matchId
0,4,87,5753,7145,0.0,0,121,0,0,183,...,3375,4744,56079,4820,5798,8492,1444,0,0,LA2_1304858815
1,60,8,5759,5296,0.0,0,73,0,10,164,...,5737,6219,64422,6218,9646,14786,271,20,1,LA2_1304858815
2,0,78,4508,6734,0.0,112,64,0,0,82,...,446,1828,41228,5862,8329,26,26,423,2,LA2_1304858815
3,0,75,6480,5214,0.0,0,63,0,0,173,...,7203,2135,41659,7888,3698,4099,375,116,3,LA2_1304858815
4,0,24,4746,4606,0.0,0,127,0,0,92,...,1344,3110,13656,3529,5639,3098,0,266,4,LA2_1304858815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47045,4,87,4570,7947,0.0,0,75,0,0,153,...,990,5530,39856,1792,5935,4955,240,405,5,LA2_1253811110
47046,60,16,5621,5078,0.0,0,72,0,0,155,...,2852,7352,72486,5316,9749,13369,245,237,6,LA2_1253811110
47047,4,73,6166,6003,0.0,0,61,0,0,196,...,7948,5402,42117,8322,8476,374,374,597,7,LA2_1253811110
47048,8,78,4723,4832,0.0,0,58,0,0,129,...,3325,2746,43180,3998,5023,398,3,0,8,LA2_1253811110


In [108]:
# Especificar la ruta y el nombre del archivo de Excel
ruta_archivo = 'data/IRON/Timeline/df_trabajo.xlsx'

# Guardar el DataFrame en un archivo de Excel
df_trabajo.to_excel(ruta_archivo, index=False)

In [89]:
combined_df.to_csv('PartidasIronTimeline.csv', index=False)

In [10]:
"""champion_stats_cl = list(df_test["championStats"][0].keys())
damage_stats_cl = list(df_test["damageStats"][0].keys())
print(champion_stats_cl)
print(damage_stats_cl)"""

['abilityHaste', 'abilityPower', 'armor', 'armorPen', 'armorPenPercent', 'attackDamage', 'attackSpeed', 'bonusArmorPenPercent', 'bonusMagicPenPercent', 'ccReduction', 'cooldownReduction', 'health', 'healthMax', 'healthRegen', 'lifesteal', 'magicPen', 'magicPenPercent', 'magicResist', 'movementSpeed', 'omnivamp', 'physicalVamp', 'power', 'powerMax', 'powerRegen', 'spellVamp']
['magicDamageDone', 'magicDamageDoneToChampions', 'magicDamageTaken', 'physicalDamageDone', 'physicalDamageDoneToChampions', 'physicalDamageTaken', 'totalDamageDone', 'totalDamageDoneToChampions', 'totalDamageTaken', 'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken']


In [6]:
#Definimos función que nos retorna un diciconario
def return_dict(diccionario, colum_names):
    lista = list(diccionario.values())
    return pd.Series(lista, index= colum_names)

In [11]:
"""df_test[champion_stats_cl] = df_test['championStats'].apply(lambda x :  return_dict(x, champion_stats_cl))
df_test[damage_stats_cl] = df_test['damageStats'].apply(lambda x :  return_dict(x, damage_stats_cl))
df_test"""

,championStats,currentGold,damageStats,goldPerSecond,jungleMinionsKilled,level,minionsKilled,participantId,position,timeEnemySpentControlled,...,magicDamageTaken,physicalDamageDone,physicalDamageDoneToChampions,physicalDamageTaken,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken
1,"{'abilityHaste': 0, 'abilityPower': 15, 'armor...",492,"{'magicDamageDone': 29266, 'magicDamageDoneToC...",0,0,12,81,1,"{'x': 3937, 'y': 13367}",257436,...,1147,15006,1174,10019,47185,8359,11287,2912,1695,120
2,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...",510,"{'magicDamageDone': 21287, 'magicDamageDoneToC...",0,69,9,2,2,"{'x': 3725, 'y': 6461}",250311,...,2317,31166,1047,8095,67902,3219,10438,15447,110,26
3,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...",74,"{'magicDamageDone': 3304, 'magicDamageDoneToCh...",0,0,10,95,3,"{'x': 11778, 'y': 3134}",0,...,3825,41612,3220,1693,48429,3985,5902,3512,476,382
4,"{'abilityHaste': 0, 'abilityPower': 20, 'armor...",547,"{'magicDamageDone': 1576, 'magicDamageDoneToCh...",0,4,9,109,4,"{'x': 6128, 'y': 6008}",369615,...,1295,52927,7393,5026,58764,7890,6321,4260,0,0
5,"{'abilityHaste': 0, 'abilityPower': 53, 'armor...",121,"{'magicDamageDone': 5947, 'magicDamageDoneToCh...",30,0,7,25,5,"{'x': 6187, 'y': 6280}",34703,...,321,5827,948,3076,16012,2319,3398,4237,90,0
6,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...",714,"{'magicDamageDone': 4200, 'magicDamageDoneToCh...",0,0,10,102,6,"{'x': 4218, 'y': 13500}",4196,...,4298,40116,7435,1898,46812,8364,7892,2495,0,1695
7,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...",521,"{'magicDamageDone': 21146, 'magicDamageDoneToC...",0,76,9,13,7,"{'x': 6170, 'y': 5781}",380395,...,2450,33335,2311,9657,79654,3531,12293,25173,168,186
8,"{'abilityHaste': 0, 'abilityPower': 80, 'armor...",358,"{'magicDamageDone': 42542, 'magicDamageDoneToC...",0,1,10,90,8,"{'x': 7165, 'y': 6678}",145479,...,1012,8805,88,3951,51347,5408,5363,0,0,400
9,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...",1442,"{'magicDamageDone': 138, 'magicDamageDoneToCha...",0,0,8,35,9,"{'x': 11436, 'y': 1588}",10199,...,750,16614,1807,4197,19248,1866,4948,2495,0,0
10,"{'abilityHaste': 0, 'abilityPower': 0, 'armor'...",602,"{'magicDamageDone': 0, 'magicDamageDoneToChamp...",20,0,8,29,10,"{'x': 6958, 'y': 6476}",29362,...,1376,10378,2171,4200,11574,2531,5666,1196,360,90


In [ ]:
#df_completo.to_excel("data/IRON/Timeline/tl_completo_ejemplo.xlsx")

In [ ]:
#tl_participant.to_excel("data/IRON/Timeline/tl_participant.xlsx")

In [ ]:
"""# Lee el archivo de Excel
excel_info = pd.read_excel('participant.xlsx', sheet_name='Sheet1')

# Aplica la condición y obtén la columna como lista ----> seleccionar las columnas del excel que establecimos como "drop"
drop_columns = excel_info.loc[excel_info['Operación'] == 'drop']['feature'].tolist()

sum_columns = excel_info.loc[excel_info['Operación'] == 'sum']['feature'].tolist()

average_columns = excel_info.loc[excel_info['Operación'] == 'average']['feature'].tolist()

bool_columns = excel_info.loc[excel_info['Operación'] == 'bool']['feature'].tolist()"""


In [ ]:
operaciones = {
    'sum': lambda x: x.sum(),
    'average': lambda x: x.mean(),
    'bool': lambda x: x.any()
}



    

"""for columna in reduced_df.columns:
    if columna in drop_columns:
        continue  # Saltar columna si se encuentra en drop_columns
    operacion = excel_info.loc[excel_info['feature'] == columna, 'Operación'].iloc[0]
    funcion = operaciones.get(operacion)
    
    if funcion is None:
        # Operación no encontrada en el diccionario, manejar el caso según sea necesario
        resultado = None  # Otra asignación o manejo adecuado
    else:
        resultado = funcion(reduced_df[columna])
    
    df_transformado[columna] = resultado"""
    
import numpy as np

# Define una función predeterminada que no hace ningún cálculo
def no_operation(x):
    return np.nan

df_transformado = pd.DataFrame(columns=reduced_df.columns)

group_size = 5
num_rows = len(reduced_df)

for i in range(0, num_rows, group_size):
    # Obetener  el grupo de filas actual (5 y 5)
    group = reduced_df.iloc[i:i+group_size]
    
    # Aplica la operación determinada para el grupo actual  
    group_result = group.apply(lambda column: operaciones.get(excel_info.loc[excel_info['feature'] == column.name, 'Operación'].iloc[0], no_operation)(column))
    
   # Agrega el resultado del grupo al DataFrame transformado
    df_transformado = df_transformado.append(group_result, ignore_index=True)



In [ ]:

df_transformado.to_csv("data/IRON/df_transformado.csv")

/tmp/ipykernel_1013/275108196.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df_trabajo.to_excel("Iron_timeline_trabajo.xls")


ModuleNotFoundError: No module named 'xlwt'

In [ ]:
columnas_tipo = {
    int: sum_columns,
    float: average_columns,
    bool: bool_columns
}

for tipo, columnas in columnas_tipo.items():
    for columna in columnas:
        df_trabajo[columna] = df_trabajo[columna].astype(tipo)

In [ ]:
df_trabajo.tail(5)

In [ ]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado "df_trabajo" con las columnas mencionadas

# Seleccionar solo las columnas numéricas
columnas_numericas = df_trabajo.select_dtypes(include=[np.number])

# Crear un nuevo DataFrame vacío con las mismas columnas que df_trabajo
df_partida_agrupada = pd.DataFrame(columns=columnas_numericas.columns)

# Obtener el número total de filas en df_trabajo
total_filas = columnas_numericas.shape[0]

# Iterar a través de las filas de columnas_numericas
for i in range(0, total_filas - 1, 2):
    # Obtener las dos filas a restar
    fila1 = columnas_numericas.iloc[i]
    fila2 = columnas_numericas.iloc[i+1]
    
    # Calcular la resta de las dos filas
    resta_filas = fila1 - fila2
    
    # Agregar la resta al nuevo DataFrame df_partida_agrupada
    df_partida_agrupada = df_partida_agrupada.append(resta_filas, ignore_index=True)

# Asignar el primer valor de las columnas booleanas en cada fila del nuevo DataFrame
for columna in df_trabajo.select_dtypes(include=[bool]):
    df_partida_agrupada[columna] = df_trabajo[columna].iloc[::2].reset_index(drop=True)

# Mostrar el resultado
print(df_partida_agrupada)


In [ ]:
# Mostrar el nuevo DataFrame
df_partida_agrupada

In [ ]:
df_partida_agrupada.to_csv("data/IRON/df_partida_agrupada.csv")

In [ ]:
columnas_tipo = {
    int: sum_columns,
    float: average_columns,
    bool: bool_columns
}

for tipo, columnas in columnas_tipo.items():
    for columna in columnas:
        df_trabajo[columna] = df_trabajo[columna].astype(tipo)
        


In [ ]:
df_trabajo.info()

In [ ]:
# Counting the number of NaN for each column
df_trabajo.isnull().sum().sort_values(ascending=False) 

In [ ]:
df_trabajo.info()

In [ ]:
df_trabajo.describe()


In [ ]:
correlation_matrix = df_trabajo.corr()
print(correlation_matrix)


In [ ]:
categorical_features = bool_columns

#Build functions
def duplicates(df):
    print(f"Duplicates droped: {df.duplicated().sum()}")
    df = df.drop_duplicates()
    return df

def missing_data_num(df_num): #TBD if needed
    return df_num

def missing_data_cat(df_cat):#TBD if needed
    return df_cat

def scale(df_num):
    #Standard or Robust if there are many otliers
    scaler = RobustScaler()
    column_names = df_num.columns.tolist()
    df_num = scaler.fit_transform(df_num)
    df_scaled = pd.DataFrame(df_num, columns=column_names)
    return df_scaled


# Start script
"""






corr_threshold = 0.9 #To be adjusted
corr_over_df = corr_df[corr_df["absolute_correlation"] >= corr_threshold]
irrelevant_features = list(set(corr_over_df["feature_2"]))
df = df.drop(columns=[irrelevant_features],inplace = True)"""

In [ ]:
#Separate categorical features
df_cat = df_trabajo[categorical_features]

#Remove missing data from categorical features
df_cat = missing_data_cat(df_cat)



In [ ]:
#Separate numerical features
df_num = df_trabajo.drop(columns=categorical_features)
#Transform numerical features
df_num = missing_data_num(df_num)
#Apply scaler to df_num
df_num_scaled = scale(df_num)
df_num_scaled


In [ ]:

#Concat categorical and numerical processed features
df_preprocessed = pd.concat([df_num_scaled,df_cat],axis=1)
df_preprocessed

In [ ]:
#correlation analysis to drop features that have high correlation
correlation_matrix = df_preprocessed.corr()
column_names = correlation_matrix.columns
correlation_matrix

In [ ]:
#pd.set_option('display.max_rows', None)
corr_df = correlation_matrix.stack().reset_index()
corr_df.columns = ['feature_1','feature_2', 'correlation']
no_self_correlation = (corr_df['feature_1'] != corr_df['feature_2'])
corr_df = corr_df[no_self_correlation]
corr_df['absolute_correlation'] = np.abs(corr_df['correlation'])
corr_df.sort_values(by="absolute_correlation", ascending=False).round(4).head(5) #Remove comment to show df



In [ ]:

df_preprocessed["win"].iloc[::2].value_counts()

In [ ]:
model = Logi

In [ ]:
corr_df.to_excel("correlation.")